In [2]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam
import math
import numpy as np
import random
from collections import deque


In [5]:
class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size # normalized previous days
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = load_model(model_name) if is_eval else self._model()
    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001))
        return model
    def act(self, state):
        if not self.is_eval and random.random()<= self.epsilon:
            return random.randrange(self.action_size)
        options = self.model.predict(state)
        return np.argmax(options[0])
    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])
        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
 

In [7]:
def formatPrice(n):
    return("-USD." if n<0 else "USD.")+"{0:.2f}".format(abs(n))
def getStockDataVec(key):
    vec = []
    lines = open(key+".csv","r").read().splitlines()
    for line in lines[1:]:
        #print(line)
        #print(float(line.split(",")[4]))
        vec.append(float(line.split(",")[4]))
        #print(vec)
    return vec 
def sigmoid(x):
    return 1/(1+math.exp(-x))
def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))
    return np.array([res])


In [25]:
import sys
stock_name = input("Enter stock_name:")
window_size = input("Enter window size:")
episode_count = input("Enter episode count:")
stock_name = str(stock_name)
window_size = int(window_size)
episode_count = int(episode_count)
agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32
for e in range(episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getState(data, 0, window_size + 1)
    total_profit = 0
    agent.inventory = []
    for t in range(l):
        action = agent.act(state)
        # sit
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0
        if action == 1: # buy
            agent.inventory.append(data[t])
            print("Buy: " + formatPrice(data[t]))
        elif action == 2 and len(agent.inventory) > 0: # sell
            bought_price = window_size_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print("--------------------------------")
            print("Total Profit: " + formatPrice(total_profit))
            print("--------------------------------")
        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)
    if e % 10 == 0:
        agent.model.save(str(e))


Enter stock_name:JPMC_train
Enter window size:20
Enter episode count:50
Episode 0/50
Buy: USD.121.86
Buy: USD.118.89
Sell: USD.118.14 | Profit: -USD.3.72
Buy: USD.119.09
Buy: USD.118.26
Sell: USD.120.09 | Profit: USD.1.20
Sell: USD.118.31 | Profit: -USD.0.78
Sell: USD.129.44 | Profit: USD.11.18
Buy: USD.131.27
Sell: USD.129.91 | Profit: -USD.1.36
Buy: USD.130.16
Sell: USD.129.73 | Profit: -USD.0.43
Buy: USD.130.07
Buy: USD.125.31
Sell: USD.119.55 | Profit: -USD.10.52
Sell: USD.115.99 | Profit: -USD.9.32
Buy: USD.112.61
Sell: USD.114.05 | Profit: USD.1.44
Buy: USD.114.72
Sell: USD.112.86 | Profit: -USD.1.86
Buy: USD.111.91
Buy: USD.108.00
Buy: USD.112.95
Buy: USD.111.79
Buy: USD.114.54
Buy: USD.113.42
Buy: USD.115.24
Sell: USD.114.81 | Profit: USD.2.90
Buy: USD.115.36
Buy: USD.112.36
Buy: USD.115.76
Buy: USD.114.35
Buy: USD.118.39
Buy: USD.122.46
Buy: USD.123.63
Buy: USD.122.59
Sell: USD.116.67 | Profit: USD.8.67
Sell: USD.115.52 | Profit: USD.2.57
Sell: USD.118.55 | Profit: USD.6.76
Bu

Buy: USD.113.73
Buy: USD.114.51
Buy: USD.113.71
Buy: USD.113.74
Buy: USD.115.90
Sell: USD.118.60 | Profit: USD.2.78
Sell: USD.119.17 | Profit: USD.6.56
Sell: USD.120.58 | Profit: USD.6.53
Sell: USD.116.39 | Profit: USD.3.77
Sell: USD.116.12 | Profit: USD.1.40
Buy: USD.117.87
Sell: USD.117.08 | Profit: USD.2.72
Sell: USD.118.16 | Profit: USD.5.19
Sell: USD.115.83 | Profit: USD.3.92
Sell: USD.111.21 | Profit: USD.3.21
Sell: USD.109.14 | Profit: -USD.3.81
Buy: USD.106.79
Buy: USD.105.85
Sell: USD.104.50 | Profit: -USD.10.06
Buy: USD.107.73
Sell: USD.112.77 | Profit: -USD.1.77
Sell: USD.110.39 | Profit: -USD.4.37
Sell: USD.108.14 | Profit: -USD.7.08
Sell: USD.105.98 | Profit: -USD.7.44
Sell: USD.104.99 | Profit: -USD.9.82
Sell: USD.101.96 | Profit: -USD.12.25
Sell: USD.103.61 | Profit: -USD.8.82
Sell: USD.109.37 | Profit: -USD.4.24
Sell: USD.111.19 | Profit: -USD.1.17
Sell: USD.115.86 | Profit: USD.0.10
Sell: USD.118.84 | Profit: USD.4.49
Sell: USD.116.51 | Profit: USD.1.13
Buy: USD.116.13

Sell: USD.141.04 | Profit: USD.1.55
Sell: USD.142.57 | Profit: -USD.0.44
Sell: USD.143.20 | Profit: USD.2.40
Sell: USD.143.80 | Profit: USD.7.23
Sell: USD.141.82 | Profit: USD.7.07
Sell: USD.142.24 | Profit: USD.7.16
Sell: USD.139.63 | Profit: USD.2.36
Sell: USD.138.56 | Profit: USD.0.11
Sell: USD.139.67 | Profit: USD.0.55
Sell: USD.140.93 | Profit: USD.0.95
Sell: USD.142.16 | Profit: USD.3.22
--------------------------------
Total Profit: USD.101.28
--------------------------------
Episode 5/50
Buy: USD.118.89
Sell: USD.118.14 | Profit: -USD.0.75
Buy: USD.119.09
Sell: USD.118.26 | Profit: -USD.0.83
Buy: USD.117.34
Sell: USD.124.60 | Profit: USD.7.26
Buy: USD.132.23
Buy: USD.129.91
Buy: USD.132.00
Sell: USD.130.16 | Profit: -USD.2.07
Buy: USD.129.73
Buy: USD.130.07
Buy: USD.128.00
Buy: USD.125.31
Buy: USD.119.55
Buy: USD.115.99
Sell: USD.114.06 | Profit: -USD.15.85
Buy: USD.115.41
Sell: USD.113.43 | Profit: -USD.18.57
Buy: USD.113.03
Buy: USD.115.83
Buy: USD.115.18
Buy: USD.115.30
Buy:

Sell: USD.127.24 | Profit: USD.8.93
Sell: USD.129.44 | Profit: USD.12.10
Sell: USD.131.27 | Profit: USD.6.67
Buy: USD.132.23
Sell: USD.129.91 | Profit: -USD.2.32
Buy: USD.132.00
Buy: USD.130.16
Buy: USD.129.73
Buy: USD.130.07
Buy: USD.128.00
Buy: USD.125.31
Buy: USD.119.55
Sell: USD.115.99 | Profit: -USD.16.01
Sell: USD.114.06 | Profit: -USD.16.10
Sell: USD.115.41 | Profit: -USD.14.32
Sell: USD.113.43 | Profit: -USD.16.64
Buy: USD.113.03
Sell: USD.115.83 | Profit: -USD.12.17
Buy: USD.115.18
Buy: USD.113.92
Buy: USD.117.32
Buy: USD.116.38
Buy: USD.115.82
Sell: USD.115.30 | Profit: -USD.10.01
Sell: USD.112.61 | Profit: -USD.6.94
Buy: USD.114.05
Sell: USD.112.62 | Profit: -USD.0.41
Sell: USD.111.89 | Profit: -USD.3.29
Sell: USD.114.72 | Profit: USD.0.80
Sell: USD.114.36 | Profit: -USD.2.96
Sell: USD.112.86 | Profit: -USD.3.52
Buy: USD.112.97
Sell: USD.111.91 | Profit: -USD.3.91
Buy: USD.108.00
Buy: USD.112.95
Sell: USD.111.79 | Profit: -USD.2.26
Sell: USD.114.56 | Profit: USD.1.59
Sell: U

Buy: USD.112.95
Buy: USD.111.79
Sell: USD.114.56 | Profit: -USD.0.74
Buy: USD.114.54
Buy: USD.115.32
Buy: USD.114.76
Buy: USD.115.22
Buy: USD.113.42
Sell: USD.115.24 | Profit: USD.2.63
Buy: USD.114.81
Sell: USD.115.36 | Profit: USD.1.31
Buy: USD.114.21
Buy: USD.112.43
Buy: USD.113.61
Buy: USD.112.36
Buy: USD.115.76
Buy: USD.114.35
Sell: USD.115.38 | Profit: USD.2.76
Buy: USD.118.39
Sell: USD.120.14 | Profit: USD.8.25
Sell: USD.122.13 | Profit: USD.7.41
Buy: USD.122.46
Buy: USD.123.63
Buy: USD.122.59
Buy: USD.121.64
Buy: USD.118.63
Buy: USD.116.67
Buy: USD.115.52
Sell: USD.115.80 | Profit: USD.1.44
Sell: USD.118.55 | Profit: USD.5.69
Sell: USD.114.67 | Profit: USD.1.70
Buy: USD.114.39
Buy: USD.114.41
Buy: USD.113.73
Buy: USD.114.51
Buy: USD.113.71
Buy: USD.113.74
Sell: USD.115.90 | Profit: USD.3.99
Sell: USD.118.60 | Profit: USD.10.60
Sell: USD.119.17 | Profit: USD.6.22
Sell: USD.120.58 | Profit: USD.8.79
Buy: USD.116.39
Buy: USD.116.12
Buy: USD.117.87
Buy: USD.117.08
Buy: USD.118.16
Bu

Buy: USD.113.42
Sell: USD.115.24 | Profit: -USD.1.14
Sell: USD.114.81 | Profit: -USD.1.01
Sell: USD.115.36 | Profit: USD.0.06
Buy: USD.114.21
Buy: USD.112.43
Buy: USD.113.61
Sell: USD.112.36 | Profit: -USD.0.25
Sell: USD.115.76 | Profit: USD.2.90
Sell: USD.114.35 | Profit: USD.1.38
Sell: USD.115.38 | Profit: USD.3.47
Sell: USD.118.39 | Profit: USD.10.39
Sell: USD.120.14 | Profit: USD.5.38
Sell: USD.122.13 | Profit: USD.8.71
Sell: USD.122.46 | Profit: USD.8.25
Sell: USD.123.63 | Profit: USD.11.20
Sell: USD.122.59 | Profit: USD.8.98
Buy: USD.118.63
Sell: USD.116.67 | Profit: -USD.1.96
Buy: USD.112.49
Buy: USD.111.21
Buy: USD.109.14
Sell: USD.106.79 | Profit: -USD.5.70
Sell: USD.105.85 | Profit: -USD.5.36
Sell: USD.107.99 | Profit: -USD.1.15
Buy: USD.104.50
Buy: USD.107.73
Sell: USD.112.77 | Profit: USD.8.27
Sell: USD.110.39 | Profit: USD.2.66
Buy: USD.108.14
Buy: USD.105.98
Buy: USD.104.99
Buy: USD.101.96
Sell: USD.103.61 | Profit: -USD.4.53
Sell: USD.109.37 | Profit: USD.3.39
Sell: USD.

Buy: USD.104.50
Sell: USD.107.73 | Profit: USD.3.23
Buy: USD.105.98
Buy: USD.104.99
Buy: USD.101.96
Sell: USD.109.37 | Profit: USD.3.39
Sell: USD.111.19 | Profit: USD.6.20
Sell: USD.115.86 | Profit: USD.13.90
Buy: USD.116.13
Sell: USD.122.23 | Profit: USD.6.10
Buy: USD.124.60
Buy: USD.126.08
Buy: USD.125.88
Buy: USD.128.15
Buy: USD.126.97
Buy: USD.127.20
Buy: USD.130.68
Buy: USD.131.37
Buy: USD.131.45
Buy: USD.129.74
Sell: USD.135.08 | Profit: USD.10.48
Sell: USD.135.30 | Profit: USD.9.22
Sell: USD.133.91 | Profit: USD.8.03
Buy: USD.132.94
Sell: USD.133.12 | Profit: USD.4.97
Buy: USD.132.54
Sell: USD.133.84 | Profit: USD.6.87
Sell: USD.133.05 | Profit: USD.5.85
Sell: USD.135.04 | Profit: USD.4.36
Sell: USD.136.48 | Profit: USD.5.11
Sell: USD.136.74 | Profit: USD.5.29
Sell: USD.134.35 | Profit: USD.4.61
Sell: USD.136.56 | Profit: USD.3.62
Sell: USD.138.18 | Profit: USD.5.64
Buy: USD.136.24
Sell: USD.135.16 | Profit: -USD.1.08
Buy: USD.130.10
Buy: USD.129.29
Buy: USD.130.06
Sell: USD.132

Sell: USD.118.84 | Profit: USD.9.70
Buy: USD.116.13
Buy: USD.122.23
Buy: USD.122.38
Buy: USD.122.70
Buy: USD.124.11
Buy: USD.124.60
Buy: USD.126.08
Buy: USD.125.88
Buy: USD.128.15
Buy: USD.126.97
Buy: USD.127.20
Buy: USD.130.68
Buy: USD.131.37
Buy: USD.131.45
Buy: USD.129.74
Buy: USD.135.08
Buy: USD.135.30
Buy: USD.133.91
Buy: USD.132.94
Buy: USD.133.12
Buy: USD.132.54
Buy: USD.133.84
Buy: USD.133.05
Buy: USD.135.04
Buy: USD.134.35
Buy: USD.136.56
Buy: USD.138.18
Buy: USD.136.24
Buy: USD.135.16
Buy: USD.131.37
Buy: USD.131.59
Sell: USD.132.88 | Profit: USD.26.09
Sell: USD.134.21 | Profit: USD.28.36
Sell: USD.134.08 | Profit: USD.26.09
Sell: USD.133.41 | Profit: USD.27.25
Sell: USD.130.10 | Profit: USD.25.60
Sell: USD.129.29 | Profit: USD.21.15
Sell: USD.130.06 | Profit: USD.24.08
Sell: USD.130.69 | Profit: USD.25.70
Sell: USD.132.16 | Profit: USD.30.20
Sell: USD.130.66 | Profit: USD.14.53
Sell: USD.131.28 | Profit: USD.9.05
Sell: USD.131.74 | Profit: USD.9.36
Sell: USD.132.46 | Profit:

Buy: USD.115.36
Buy: USD.114.21
Sell: USD.112.43 | Profit: -USD.2.93
Sell: USD.113.61 | Profit: -USD.0.60
Buy: USD.118.16
Buy: USD.115.83
Buy: USD.112.49
Sell: USD.111.21 | Profit: -USD.6.95
Sell: USD.109.14 | Profit: -USD.6.69
Sell: USD.106.79 | Profit: -USD.5.70
Buy: USD.131.37
Buy: USD.131.45
Buy: USD.129.74
Buy: USD.135.08
Sell: USD.135.30 | Profit: USD.3.93
Sell: USD.133.91 | Profit: USD.2.46
Sell: USD.132.94 | Profit: USD.3.20
Sell: USD.133.12 | Profit: -USD.1.96
Buy: USD.142.16
Buy: USD.143.35
Sell: USD.142.55 | Profit: USD.0.39
Sell: USD.141.07 | Profit: -USD.2.28
--------------------------------
Total Profit: -USD.18.00
--------------------------------
Episode 24/50
Buy: USD.117.32
Buy: USD.116.38
Sell: USD.115.82 | Profit: -USD.1.50
Sell: USD.115.30 | Profit: -USD.1.08
Buy: USD.123.63
Buy: USD.122.59
Sell: USD.121.64 | Profit: -USD.1.99
Sell: USD.118.63 | Profit: -USD.3.96
Buy: USD.112.77
Buy: USD.110.39
Buy: USD.108.14
Sell: USD.105.98 | Profit: -USD.6.79
Sell: USD.104.99 | 

Sell: USD.118.14 | Profit: -USD.0.75
Buy: USD.113.03
Buy: USD.115.83
Sell: USD.115.18 | Profit: USD.2.15
Sell: USD.113.92 | Profit: -USD.1.91
Buy: USD.115.80
Buy: USD.118.55
Sell: USD.114.67 | Profit: -USD.1.13
Sell: USD.114.39 | Profit: -USD.4.16
Buy: USD.111.19
Buy: USD.115.86
Sell: USD.118.84 | Profit: USD.7.65
Sell: USD.116.51 | Profit: USD.0.65
Buy: USD.124.11
Sell: USD.124.60 | Profit: USD.0.49
Buy: USD.135.08
Buy: USD.137.27
Sell: USD.138.45 | Profit: USD.3.37
Sell: USD.139.12 | Profit: USD.1.85
--------------------------------
Total Profit: USD.8.21
--------------------------------
Episode 38/50
Buy: USD.129.73
Buy: USD.130.07
Sell: USD.128.00 | Profit: -USD.1.73
Sell: USD.125.31 | Profit: -USD.4.76
Buy: USD.112.36
Buy: USD.115.76
Sell: USD.114.35 | Profit: USD.1.99
Sell: USD.115.38 | Profit: -USD.0.38
Buy: USD.105.98
Buy: USD.104.99
Sell: USD.101.96 | Profit: -USD.4.02
Sell: USD.103.61 | Profit: -USD.1.38
Buy: USD.129.74
Sell: USD.135.08 | Profit: USD.5.34
Buy: USD.143.65
Buy:

In [37]:
stock_name = input("Enter Stock_name:")
model_name = '10'
model = load_model(model_name)
window_size = model.layers[0].input.shape.as_list()[1]
agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)
print(data)
l = len(data) - 1
batch_size = 32
state = getState(data, 0, window_size + 1)
print(state)
total_profit = 0
agent.inventory = []
print(l)
for t in range(l):
    action = agent.act(state)
    print(action)
    # sit
    next_state = getState(data, t + 1, window_size + 1)
    reward = 0
    if action == 1: # buy
        agent.inventory.append(data[t])
        print("Buy: " + formatPrice(data[t]))
    elif action == 2 and len(agent.inventory) > 0: # sell
        bought_price = agent.inventory.pop(0)
        reward = max(data[t] - bought_price, 0)
        total_profit += data[t] - bought_price
        print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
    done = True if t == l - 1 else False
    agent.memory.append((state, action, reward, next_state, done))
    state = next_state
    if done:
        print("--------------------------------")
        print("Profit from trading one unit of stock:" + formatPrice(total_profit))
        print("--------------------------------")
        print ("Total profit from 100 units of stock is:",formatPrice(total_profit*100))
 

Enter Stock_name:jpmc_apr_may
[138.619995, 137.800003, 130.339996, 133.649994, 131.25, 134.619995, 128.259995, 130.75, 125.809998, 127.139999, 130.550003, 127.18, 126.839996, 124.910004, 128.490005, 128.880005, 129.139999, 128.75, 130.309998, 130.160004, 128.419998, 127.610001, 127.470001, 127.889999, 128.520004, 128.5, 128.990005, 138.729996, 139.830002, 141.399994, 141.220001, 140.809998, 140.539993, 140.729996, 137.669998, 135.229996, 137.050003, 138.240005, 141.199997, 138.919998, 135.979996, 134.119995, 136.740005]
[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5]]
42
2
2
1
Buy: USD.130.34
2
Sell: USD.133.65 | Profit: USD.3.31
2
2
1
Buy: USD.128.26
2
Sell: USD.130.75 | Profit: USD.2.49
1
Buy: USD.125.81
2
Sell: USD.127.14 | Profit: USD.1.33
2
2
1
Buy: USD.126.84
1
Buy: USD.124.91
2
Sell: USD.128.49 | Profit: USD.1.65
2
Sell: USD.128.88 | Profit: USD.3.97
2
2
2
1
Buy: USD.130.16
2
Sell: USD.128.42 | Profit: -USD.1.74
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
